# Sentimental model

i have downloaded the data from this site #"http://thinknook.com/wp-content/uploads/2012/09/Sentiment-Analysis-Dataset.zip"

In [1]:
import pandas as pd
import numpy as np
import spacy


In [ ]:
%%bash 
wget http://thinknook.com/wp-content/uploads/2012/09/Sentiment-Analysis-Dataset.zip -nc -P /.

In [2]:
df = pd.read_csv("./sentiment.csv", error_bad_lines = False)

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


In [3]:
df.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [4]:
df['SentimentText'][:10]

0                         is so sad for my APL frie...
1                       I missed the New Moon trail...
2                              omg its already 7:30 :O
3              .. Omgaga. Im sooo  im gunna CRy. I'...
4             i think mi bf is cheating on me!!!   ...
5                    or i just worry too much?        
6                   Juuuuuuuuuuuuuuuuussssst Chillin!!
7           Sunny Again        Work Tomorrow  :-|  ...
8          handed in my uniform today . i miss you ...
9             hmmmm.... i wonder how she my number @-)
Name: SentimentText, dtype: object

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk import TweetTokenizer

In [7]:
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True)


In [8]:
tweet_tokenizer.tokenize(df.iloc[0, -1])

['is', 'so', 'sad', 'for', 'my', 'apl', 'friend', '...']

In [10]:
df.iloc[0, -1]

'                     is so sad for my APL friend.............'

In [11]:
from html import unescape
def preprocessor(doc):
    return unescape(doc).lower()

In [13]:
unescape("&lt")

'<'

In [14]:
nlp = spacy.load("en")

In [16]:
nlp = spacy.load("en", disable = ["ner", "paser", "tagger"])

In [17]:
def lemmatizer(doc):
    return [word.lemma_ for word in nlp(doc)]

In [18]:
from spacy.lang.en import STOP_WORDS

In [29]:
STOP_WORDS_lemma = [word.lemma_ for word in nlp(" ".join(list(STOP_WORDS)))]
STOP_WORDS_lemma = set(STOP_WORDS_lemma).union({",", ".", ";"})

In [26]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [30]:
vectorizer = TfidfVectorizer(preprocessor=preprocessor,
                            tokenizer = lemmatizer,
                            ngram_range = (1,2),
                            stop_words = STOP_WORDS_lemma)

clf = MultinomialNB()
pipe = Pipeline([("vectorizer", vectorizer), ("classifier", clf)])

In [31]:
X = df["SentimentText"]
y = df["Sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0 )

In [ ]:
pipe.fit(X_train, y_train)